# Lab 5 Text Clasification

Roshan Ahmed
20BRS1072

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip /content/smsspamcollection.zip

--2023-06-07 15:58:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 502 Bad Gateway
2023-06-07 15:58:30 ERROR 502: Bad Gateway.

Archive:  /content/smsspamcollection.zip
replace SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import torchtext
import torch
from torch import nn,optim
from time import time

In [ ]:
data = pd.read_csv("/content/SMSSpamCollection",delimiter = "	",names = ["sh","text"])

data['label'] = z
data = data.drop('sh',axis = 1)
print(data)
data.to_csv("messages.csv")


                                                   text  label
0     Go until jurong point, crazy.. Available only ...      0
1                         Ok lar... Joking wif u oni...      0
2     Free entry in 2 a wkly comp to win FA Cup fina...      1
3     U dun say so early hor... U c already then say...      0
4     Nah I don't think he goes to usf, he lives aro...      0
...                                                 ...    ...
5567  This is the 2nd time we have tried 2 contact u...      1
5568               Will ü b going to esplanade fr home?      0
5569  Pity, * was in mood for that. So...any other s...      0
5570  The guy did some bitching but I acted like i'd...      0
5571                         Rofl. Its true to its name      0

[5572 rows x 2 columns]


Upsampling minority class

In [ ]:
spam = data[data.label==1]

data = pd.concat([data,spam,spam,spam,spam])

print(data)
print( data[data.label==1])

                                                   text  label
0     Go until jurong point, crazy.. Available only ...      0
1                         Ok lar... Joking wif u oni...      0
2     Free entry in 2 a wkly comp to win FA Cup fina...      1
3     U dun say so early hor... U c already then say...      0
4     Nah I don't think he goes to usf, he lives aro...      0
...                                                 ...    ...
5537  Want explicit SEX in 30 secs? Ring 02073162414...      1
5540  ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...      1
5547  Had your contract mobile 11 Mnths? Latest Moto...      1
5566  REMINDER FROM O2: To get 2.50 pounds free call...      1
5567  This is the 2nd time we have tried 2 contact u...      1

[8560 rows x 2 columns]
                                                   text  label
2     Free entry in 2 a wkly comp to win FA Cup fina...      1
5     FreeMsg Hey there darling it's been 3 week's n...      1
8     WINNER!! As a valued net

Creating tokenizer and vocabulary

In [ ]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn.utils import rnn
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)


tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        #print(dataset)
        for text in dataset['text']:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([data,data]), min_freq=1, specials=["<UNK>"])

vocab.set_default_index(vocab["<UNK>"])

for l in data['text']:
  tokens = tokenizer(l)
  indexes = vocab(tokens)


Creating Dataloader

In [ ]:

class Text(Dataset):
  def __init__(self,df):
    self.df = df
    #print(df)

  def pad(self,seq):
    if len(seq)>50:
      return seq[:50]
    else:
      for i in range(50-len(seq)):
        seq.insert(0,0)
      return seq

  def __len__(self):
    return len(self.df['label'])
  def __getitem__(self,idx):
    return torch.tensor(self.pad(vocab(tokenizer(self.df['text'][idx])))) , self.df['label'][idx]

test_loader  = DataLoader(Text(test.reset_index()) , batch_size=4)
train_loader = DataLoader(Text(train.reset_index()) , batch_size=64)

for txt , tgt in test_loader:
  _
  #print(txt)
  #print(txt)
voc = len(vocab)
#print(tokens)

print(vocab(["<UNK>"]))

[0]


In [ ]:
print(test)
print(train)

                                                   text  label
4599  I'm stuck in da middle of da row on da right h...      0
4008                        I'm reaching home in 5 min.      0
1475  Friendship is not a game to play, It is not a ...      0
3844  Yes ammae....life takes lot of turns you can o...      0
4536                      IM LATE TELLMISS IM ON MY WAY      0
...                                                 ...    ...
305   SMS. ac Blind Date 4U!: Rodds1 is 21/m from Ab...      1
333   Call Germany for only 1 pence per minute! Call...      1
132             Dear, will call Tmorrow.pls accomodate.      0
1350  FREE2DAY sexy St George's Day pic of Jordan!Tx...      1
3377  Good afternon, my love. How are today? I hope ...      0

[1712 rows x 2 columns]
                                                   text  label
4754  Cashbin.co.uk (Get lots of cash this weekend!)...      1
1322  I wake up long ago already... Dunno, what othe...      0
334   Any chance you might hav

Creating RNN

In [ ]:

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.embedding_layer = nn.Embedding(len(vocab),32)
        self.rnn = nn.RNN(32,16, 4, batch_first=True)
        self.linear1 = nn.Linear(16,8)
        self.linear2 = nn.Linear(8,2)
        self.act = nn.ReLU()
        self.out = nn.LogSoftmax(dim = 1)

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch)
        output, hidden = self.rnn(embeddings)
        return self.out(self.linear2(self.act(self.linear1(self.act(output[:,-1])))))

model = RNN()

print(model)

RNN(
  (embedding_layer): Embedding(9198, 32)
  (rnn): RNN(32, 16, num_layers=4, batch_first=True)
  (linear1): Linear(in_features=16, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=2, bias=True)
  (act): ReLU()
  (out): LogSoftmax(dim=1)
)


In [ ]:
accuracy = []
optimizer = optim.SGD(model.parameters(), lr=0.007, momentum=0.90)
criterion = nn.NLLLoss()
time0 = time()

Training

In [ ]:
epochs = 20
for e in range(epochs):
    running_loss = 0
    for text, tgt in train_loader:
        # Training pass
        optimizer.zero_grad()

        output = model(text)

        loss = criterion(output, tgt)

        #This is where the model learns by backpropagating
        loss.backward()
        #And optimizes its weights here
        optimizer.step()
        #print(loss.item())
        running_loss += loss.item()

    else:
        #print("\nTraining Time (in minutes) =",(time()-time0)/60)
        #print(output)
        print("\nEpoch {} - Training loss: {}".format(e+1, running_loss/len(test_loader)))
    correct_count, all_count = 0, 0
    for images,labels in test_loader:
      for i in range(len(labels)):
        img = images[i].view(1,-1)
        with torch.no_grad():
          logps = model(img)


        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        #print(pred_label)
        if(true_label == pred_label):
          correct_count += 1
        all_count += 1

  #print("Number Of Images Tested =", all_count)
    print("Model Accuracy =", (correct_count/all_count))



Epoch 1 - Training loss: 0.17151256782986293
Model Accuracy = 0.5817757009345794

Epoch 2 - Training loss: 0.16969944007485827
Model Accuracy = 0.5811915887850467

Epoch 3 - Training loss: 0.16401223643360852
Model Accuracy = 0.7009345794392523

Epoch 4 - Training loss: 0.13987906367700792
Model Accuracy = 0.834696261682243

Epoch 5 - Training loss: 0.0921846120574764
Model Accuracy = 0.852803738317757

Epoch 6 - Training loss: 0.07627979702551231
Model Accuracy = 0.9018691588785047

Epoch 7 - Training loss: 0.06716198218272668
Model Accuracy = 0.9065420560747663

Epoch 8 - Training loss: 0.06011618886177785
Model Accuracy = 0.9065420560747663

Epoch 9 - Training loss: 0.0540617302791259
Model Accuracy = 0.9182242990654206

Epoch 10 - Training loss: 0.044291950535969196
Model Accuracy = 0.9404205607476636

Epoch 11 - Training loss: 0.03954624881637152
Model Accuracy = 0.9246495327102804

Epoch 12 - Training loss: 0.034245613087664974
Model Accuracy = 0.9415887850467289

Epoch 13 - Tra

#Texts for testing:

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's

Even my brother is not like to speak with me. They treat me like aids patent.



In [ ]:
test = input()
tokens = tokenizer(test)
indexes = vocab(tokens)
logps = model(torch.tensor(indexes).view(1,-1))
ps = torch.exp(logps)
probab = list(ps.detach().numpy()[0])
pred_label = probab.index(max(probab))
if pred_label == 0:
  print("normal")
else:
  print("spam")

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
spam
